<a href="https://colab.research.google.com/github/Pakhi27/Bi-Directional-LSTM-RNN-Fake-News-Classifier/blob/main/FakeNewsClassifier_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
def handle_bad_line(line):
    print(f"Bad line: {line}")
    return None  # or return the line modified to fit the expected format

df = pd.read_csv('train.csv', delimiter=',', encoding='utf-8', on_bad_lines=handle_bad_line, engine='python')



In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(10721, 5)

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

,0
id,0
title,292
author,1017
text,22
label,0


In [8]:
###Drop Nan Values
df=df.dropna()

In [9]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [10]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [12]:
## Get the Dependent features
y=df['label']

In [13]:
X.shape

(9412, 4)

In [14]:
y.shape

(9412,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.17.0'

In [17]:
from tensorflow.keras.layers import Embedding# word 2 vec
from tensorflow.keras.preprocessing.sequence import pad_sequences# pre-padding and post padding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
### Vocabulary size
voc_size=5000

In [ ]:
# One Hot representation

In [19]:
messages=X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
10714,10714,How they could steal the election from Trump e...,Dr. Patrick Slattery,How they could steal the election from Trump e...
10715,10715,Republican Men Say It’s a Better Time to Be a ...,Claire Cain Miller,To be a woman in the United States is to feel ...
10716,10716,Border Patrol Agents Arrest Two Illegal Worker...,Bob Price,Border Patrol agents in Vermont arrested two i...
10717,10717,DNC Senior Adviser: Trump ’So Emasculated’ by ...,Ian Hanchett,On Thursday’s broadcast of the Fox News Channe...


In [22]:
messages.reset_index(inplace=True)

In [23]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
9407,10714,10714,How they could steal the election from Trump e...,Dr. Patrick Slattery,How they could steal the election from Trump e...
9408,10715,10715,Republican Men Say It’s a Better Time to Be a ...,Claire Cain Miller,To be a woman in the United States is to feel ...
9409,10716,10716,Border Patrol Agents Arrest Two Illegal Worker...,Bob Price,Border Patrol agents in Vermont arrested two i...
9410,10717,10717,DNC Senior Adviser: Trump ’So Emasculated’ by ...,Ian Hanchett,On Thursday’s broadcast of the Fox News Channe...


In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
# stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # removing special characters and replacing it with blanks
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2083, 4222, 1520, 2598, 3026, 875, 4360, 993, 4275, 4048],
 [1990, 2408, 342, 283, 3620, 1823, 4202],
 [984, 1637, 3391, 1324],
 [2149, 3447, 4178, 1935, 1330, 3843],
 [3687, 3620, 2546, 4265, 2365, 810, 3620, 4563, 2882, 176],
 [1996,
  1235,
  878,
  1103,
  266,
  781,
  4674,
  3716,
  4918,
  3053,
  3408,
  2848,
  4831,
  2836,
  4202],
 [4795, 679, 4541, 1708, 376, 3493, 1830, 4963, 2099, 688, 2113],
 [3531, 2723, 3874, 3476, 3229, 4312, 781, 2578, 2099, 688, 2113],
 [979, 2946, 3096, 4727, 3444, 2897, 4658, 1414, 781, 4385],
 [1110, 4812, 2645, 1830, 2782, 482, 730, 1489],
 [2565, 449, 2480, 3429, 3582, 3211, 3134, 2003, 3348, 2497, 4113],
 [1935, 4084, 3026, 2897, 781, 3229],
 [4468, 4695, 1339, 3132, 1564, 2607, 1179, 4639, 4742],
 [4869, 1033, 883, 3350, 1669, 4748, 1745, 2099, 688, 2113],
 [4292, 1317, 2871, 1657, 3786, 2099, 688, 2113],
 [4631, 891, 1439, 2171, 3911, 4658, 1858, 2629, 518, 4290],
 [1413, 613, 2408],
 [4190, 774, 109, 3898, 781, 2569, 2660, 4202],
 [4721

In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [31]:
onehot_repr[1]

[1990, 2408, 342, 283, 3620, 1823, 4202]

In [ ]:
# Embedding representation

In [32]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2083 4222 1520 ...    0    0    0]
 [1990 2408  342 ...    0    0    0]
 [ 984 1637 3391 ...    0    0    0]
 ...
 [3844 4776 3582 ...    0    0    0]
 [4208 1402  493 ...    0    0    0]
 [2361 3220 2807 ...    0    0    0]]


In [33]:
embedded_docs[1]

array([1990, 2408,  342,  283, 3620, 1823, 4202,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [34]:
embedded_docs[0]

array([2083, 4222, 1520, 2598, 3026,  875, 4360,  993, 4275, 4048,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [35]:
## Creating model
# each and every word is going to get converted into a vector of 40 size
embedding_vector_features=40 ##features representation
model=Sequential()

# embedding layer
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

# LSTM-100 NEURONS
model.add(LSTM(100))

# Sigmoid for binary prediction in model
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Assuming voc_size and sent_length are predefined variables
embedding_vector_features = 40  # Size of the embedding vector

model = Sequential()

# Embedding layer with correct input_dim (voc_size) and without deprecated input_length
model.add(Embedding(input_dim=voc_size, output_dim=embedding_vector_features))

# LSTM layer
model.add(LSTM(100))

# Dense layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [37]:
model.build(input_shape=(None, sent_length))


In [38]:
import numpy as np

# Dummy data: batch size of 1, sentence length of sent_length
dummy_input = np.random.randint(0, voc_size, (1, sent_length))
model.predict(dummy_input)

print(model.summary())

# Embedding Layer
# 20: This is the input length or sequence length, which represents the number of words in each input sequence.
# 40: This is the embedding dimension size (embedding_vector_features), which is the size of each word's embedding vector.
# This is the total number of parameters in the Embedding layer.
# Calculated as voc_size * embedding_vector_features = 5000 * 40 = 200,000.

# LSTM LAYER
# None: Again, the batch size is flexible.
# 100: This is the number of LSTM units (neurons) in the layer.
# This is the total number of parameters in the LSTM layer.
# The LSTM parameters include:
# 4 * [(embedding_vector_features + LSTM_units) * LSTM_units + LSTM_units]
# Specifically: 4 * [(40 + 100) * 100 + 100] = 4 * [140 * 100 + 100] = 4 * [14,000 + 100] = 4 * 14,100 = 56,400.
# These parameters include the weights for input, forget, cell, and output gates in the LSTM.

# Dense Layer
# Output Shape: (None, 1)
# None: Again, the batch size is flexible.
# 1: This is the output size, which is 1 because the model is set up for binary classification (predicting one of two classes).
# Param # (101):
# This is the total number of parameters in the Dense layer.
# Calculated as LSTM_units + 1 = 100 + 1 = 101.

# Total Parameters:
# Total params: 256,501

# This is the sum of all the parameters across all layers: 200,000 (Embedding) + 56,400 (LSTM) + 101 (Dense) = 256,501.
# Trainable params: 256,501

# All the parameters in the model are trainable, meaning they will be updated during training to minimize the loss.
# Non-trainable params: 0

# There are no non-trainable parameters in this model. Non-trainable parameters might exist in models with layers like Batch Normalization where some parameters are not updated during training.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 404ms/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [39]:
len(embedded_docs),y.shape

(9412, (9412,))

In [40]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [41]:
X_final.shape,y_final.shape

((9412, 20), (9412,))

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
# Model Training

In [43]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.6908 - loss: 0.5128 - val_accuracy: 0.9166 - val_loss: 0.2113
Epoch 2/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.9497 - loss: 0.1363 - val_accuracy: 0.9176 - val_loss: 0.2064
Epoch 3/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.9816 - loss: 0.0719 - val_accuracy: 0.9214 - val_loss: 0.2216
Epoch 4/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9915 - loss: 0.0396 - val_accuracy: 0.9182 - val_loss: 0.2869
Epoch 5/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.9942 - loss: 0.0261 - val_accuracy: 0.9131 - val_loss: 0.4078
Epoch 6/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9941 - loss: 0.0215 - val_accuracy: 0.9221 - val_loss: 0.2709
Epoch 7/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.9952 - loss: 0.0201 - val_accuracy: 0.9192 - val_loss: 0.3691
Epoch 8/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.9936 - loss: 0.0202 - val_accuracy: 0.9144 - v

In [ ]:
# Performance Metrics & Accuracy

In [44]:
y_pred=model.predict(X_test)

98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [45]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve

In [46]:
# setting a threshold value of 0.5->0.5=1 and <0.5 =0
from sklearn.metrics import confusion_matrix

In [47]:
confusion_matrix(y_test,y_pred)

array([[1641,  149],
       [ 128, 1188]])

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)
#91%

0.9108177720540889

In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92      1790
           1       0.89      0.90      0.90      1316

    accuracy                           0.91      3106
   macro avg       0.91      0.91      0.91      3106
weighted avg       0.91      0.91      0.91      3106



In [ ]:
# Bidirectional LSTM RNN

In [50]:
from tensorflow.keras.layers import Bidirectional

In [52]:
embedding_vector_features=40 ##features representation
model=Sequential()

# embedding layer
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

# LSTM-100 NEURONS
model.add(Bidirectional(LSTM(200)))

# Sigmoid for binary prediction in model
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [53]:
model.build(input_shape=(None, sent_length))

In [54]:
import numpy as np

# Dummy data: batch size of 1, sentence length of sent_length
dummy_input = np.random.randint(0, voc_size, (1, sent_length))
model.predict(dummy_input)

print(model.summary())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 400)                 │         385,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             401 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 586,001 (2.24 MB)

 Trainable params: 586,001 (2.24 MB)

 Non-trainable params: 0 (0.00 B)

None


In [55]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
# Model Training

In [57]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 24s 185ms/step - accuracy: 0.7303 - loss: 0.4841 - val_accuracy: 0.9182 - val_loss: 0.1924
Epoch 2/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 19s 170ms/step - accuracy: 0.9524 - loss: 0.1249 - val_accuracy: 0.9256 - val_loss: 0.1907
Epoch 3/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 17s 169ms/step - accuracy: 0.9789 - loss: 0.0656 - val_accuracy: 0.9198 - val_loss: 0.2479
Epoch 4/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 19s 156ms/step - accuracy: 0.9910 - loss: 0.0281 - val_accuracy: 0.9070 - val_loss: 0.2490
Epoch 5/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 22s 168ms/step - accuracy: 0.9922 - loss: 0.0335 - val_accuracy: 0.9137 - val_loss: 0.3494
Epoch 6/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 20s 165ms/step - accuracy: 0.9950 - loss: 0.0175 - val_accuracy: 0.9231 - val_loss: 0.4023
Epoch 7/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 21s 173ms/step - accuracy: 0.9976 - loss: 0.0091 - val_accuracy: 0.9198 - val_loss: 0.3730
Epoch 8/10
99/99 ━━━━━━━━━━━━━━━━━━━━ 20s 173ms/step - accuracy: 0.9971 - loss: 0.0101 - val_accu

In [58]:
# Performance Metrics & Accuracy
y_pred=model.predict(X_test)

98/98 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step


In [60]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve
y_pred

array([[0],
       [1],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [61]:
# setting a threshold value of 0.5->0.5=1 and <0.5 =0
from sklearn.metrics import confusion_matrix

In [62]:
confusion_matrix(y_test,y_pred)

array([[1650,  140],
       [ 118, 1198]])

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9169349645846748

In [64]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      1790
           1       0.90      0.91      0.90      1316

    accuracy                           0.92      3106
   macro avg       0.91      0.92      0.92      3106
weighted avg       0.92      0.92      0.92      3106

